In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd

# Reading the csv file
df = pd.read_csv('/content/drive/MyDrive/RansomwareData.csv')


Mounted at /content/drive/


In [ ]:
X = df.drop(df.columns[0], axis=1)
X = X.drop(df.columns[1], axis=1)
X = X.drop(df.columns[2], axis=1)
y = df[df.columns[2]]

y_binary = df[df.columns[1]]

# Create group labels
def convert_to_group(label):
    if 1 <= label <= 3:
        return 1
    elif 4 <= label <= 6:
        return 2
    elif 7 <= label <= 9:
        return 3
    elif 10 <= label <= 12:
        return 4
    else:
        return 0  # Assuming 0 is for goodware

y_group = y.apply(convert_to_group)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, y_train_binary, y_test_binary, y_train_group, y_test_group = train_test_split(X, y, y_binary, y_group, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Define the autoencoder with a larger encoding dimension
input_dim = X_train.shape[1]
encoding_dim = 64  # Increased dimension

# Encoder
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)

# Decoder
decoder = Dense(input_dim, activation="sigmoid")(encoder)

# Autoencoder
autoencoder = Model(inputs=input_layer, outputs=decoder)
#SDG
# Compile the autoencoder with a lower learning rate
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))


Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 125ms/step - loss: 0.1924 - val_loss: 0.0185
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - loss: 0.0128 - val_loss: 0.0073
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 147ms/step - loss: 0.0059 - val_loss: 0.0050
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 153ms/step - loss: 0.0037 - val_loss: 0.0042
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - loss: 0.0034 - val_loss: 0.0037
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - loss: 0.0030 - val_loss: 0.0034
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 125ms/step - loss: 0.0028 - val_loss: 0.0033
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - loss: 0.0027 - val_loss: 0.0031
Epoch 9/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - loss: 0.0028 - val_loss: 0.0030
Epoch 10/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 127ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 11/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - loss: 0.0023 - val_loss: 0.0029
Epoch 12/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - 

In [ ]:
# Extract the encoder part of the autoencoder
encoder_model = Model(inputs=input_layer, outputs=encoder)

# Transform the data to encoded representations
X_train_encoded = encoder_model.predict(X_train)
X_test_encoded = encoder_model.predict(X_test)


39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import pandas as pd

# Ensure input dimensions are correct
encoding_dim = X_train_encoded.shape[1]

# Input layer
input_layer = Input(shape=(encoding_dim,))

# Shared layers
x = Dense(64, activation='relu')(input_layer)
x = Dense(64, activation='relu')(x)  # Fixed duplicate use of input_layer

# Binary Classification Head (Goodware vs Malicious)
binary_output = Dense(1, activation='sigmoid', name='binary_output')(x)

# Group Classification Head
group_output = Dense(5, activation='softmax', name='group_output')(x)

# Specific Classification Head
specific_output = Dense(12, activation='softmax', name='specific_output')(x)

# Create the model
model = Model(inputs=input_layer, outputs=[binary_output, group_output, specific_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'binary_output': 'binary_crossentropy', 'group_output': 'categorical_crossentropy', 'specific_output': 'categorical_crossentropy'},
              metrics={'binary_output': 'accuracy', 'group_output': 'accuracy', 'specific_output': 'accuracy'})

# Summary of the model
model.summary()

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

label_to_group = {
    0: 0,  # Goodware
    1: 1,  # Critroni
    2: 1,  # CryptLocker
    3: 1,  # CryptoWall
    4: 2,  # KOLLAH
    5: 2,  # Kovter
    6: 2,  # Locker
    7: 3,  # MATSNU
    8: 3,  # PGPCODER
    9: 3,  # Reveton
    10: 4,  # TeslaCrypt
    11: 4,  # Trojan-Ransom
}
# Assuming y_train and y_test are pandas Series
y_train_group = y_train.map(label_to_group)
y_test_group = y_test.map(label_to_group)

# Binary labels (Goodware vs Malicious)
y_train_binary = (y_train > 0).astype(int)
y_test_binary = (y_test > 0).astype(int)

# One-hot encode the group and specific labels
y_train_group_categorical = to_categorical(y_train_group, num_classes=5)
y_train_categorical = to_categorical(y_train, num_classes=12)
y_test_group_categorical = to_categorical(y_test_group, num_classes=5)
y_test_categorical = to_categorical(y_test, num_classes=12)

# Train the model
history = model.fit(X_train_encoded,
                    {'binary_output': y_train_binary, 'group_output': y_train_group_categorical, 'specific_output': y_train_categorical},
                    epochs=500,
                    batch_size=4,
                    validation_data=(X_test_encoded, {'binary_output': y_test_binary, 'group_output': y_test_group_categorical, 'specific_output': y_test_categorical}),
                    callbacks=[early_stopping])


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 64)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 64)             │          4,160 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 64)             │          4,160 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │             65 │ dense_17[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ group_output (Dense)      │ (None, 5)              │            325 │ dense_17[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ specific_output (Dense)   │ (None, 12)             │            780 │ dense_17[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,490 (37.07 KB)

 Trainable params: 9,490 (37.07 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
305/305 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - binary_output_accuracy: 0.7806 - group_output_accuracy: 0.5894 - loss: 3.5068 - specific_output_accuracy: 0.5748 - val_binary_output_accuracy: 0.8557 - val_group_output_accuracy: 0.7115 - val_loss: 2.0327 - val_specific_output_accuracy: 0.7148
Epoch 2/500
305/305 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - binary_output_accuracy: 0.8761 - group_output_accuracy: 0.6796 - loss: 2.1358 - specific_output_accuracy: 0.6752 - val_binary_output_accuracy: 0.8689 - val_group_output_accuracy: 0.7311 - val_loss: 1.9929 - val_specific_output_accuracy: 0.7049
Epoch 3/500
305/305 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - binary_output_accuracy: 0.8672 - group_output_accuracy: 0.7121 - loss: 2.1058 - specific_output_accuracy: 0.6890 - val_binary_output_accuracy: 0.8885 - val_group_output_accuracy: 0.6918 - val_loss: 1.9658 - val_specific_output_accuracy: 0.7279
Epoch 4/500
305/305 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - binary_output_accuracy: 0.8758 - group_output_ac

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Generate predictions
predictions = model.predict(X_test_encoded)

# Binary predictions and metrics
binary_predictions = (predictions[0] > 0.5).astype(int)  # Convert probabilities to 0 or 1
binary_accuracy = accuracy_score(y_test_binary, binary_predictions)
binary_precision = precision_score(y_test_binary, binary_predictions)
binary_recall = recall_score(y_test_binary, binary_predictions)
binary_f1 = f1_score(y_test_binary, binary_predictions)
# Specific predictions and metrics
specific_predictions = np.argmax(predictions[2], axis=1)
specific_accuracy = accuracy_score(y_test, specific_predictions)
specific_precision = precision_score(y_test, specific_predictions, average='macro')
specific_recall = recall_score(y_test, specific_predictions, average='macro')
specific_f1 = f1_score(y_test, specific_predictions, average='macro')

# Print the results
print(f"Binary Classification Metrics:\n"
      f"Accuracy: {binary_accuracy:.4f}\n"
      f"Precision: {binary_precision:.4f}\n"
      f"Recall: {binary_recall:.4f}\n"
      f"F1 Score: {binary_f1:.4f}")

print(f"\nSpecific Classification Metrics:\n"
      f"Accuracy: {specific_accuracy:.4f}\n"
      f"Precision: {specific_precision:.4f}\n"
      f"Recall: {specific_recall:.4f}\n"
      f"F1 Score: {specific_f1:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Binary Classification Metrics:
Accuracy: 0.9541
Precision: 0.9558
Recall: 0.9231
F1 Score: 0.9391

Specific Classification Metrics:
Accuracy: 0.7738
Precision: 0.3640
Recall: 0.3387
F1 Score: 0.3414


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import matplotlib.pyplot as plt

# Extract the history for each output
binary_loss = history.history['binary_output_loss']
val_binary_loss = history.history['val_binary_output_loss']
specific_loss = history.history['specific_output_loss']
val_specific_loss = history.history['val_specific_output_loss']

binary_acc = history.history['binary_output_accuracy']
val_binary_acc = history.history['val_binary_output_accuracy']
specific_acc = history.history['specific_output_accuracy']
val_specific_acc = history.history['val_specific_output_accuracy']

# Plot loss
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(binary_loss, linestyle='--', label='Binary Output Loss')
plt.plot(val_binary_loss, label='Val Binary Output Loss')
plt.plot(specific_loss, linestyle='--', label='Specific Output Loss')
plt.plot(val_specific_loss, label='Val Specific Output Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(2, 1, 2)
plt.plot(binary_acc, linestyle='--', label='Binary Output Accuracy')
plt.plot(val_binary_acc, label='Val Binary Output Accuracy')
plt.plot(group_acc, linestyle='--', label='Group Output Accuracy')
plt.plot(val_group_acc, label='Val Group Output Accuracy')
plt.plot(specific_acc, linestyle='--', label='Specific Output Accuracy')
plt.plot(val_specific_acc, label='Val Specific Output Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Predict on the test set
predictions = model.predict(X_test_encoded)

# Binary classification confusion matrix
binary_preds = np.round(predictions[0]).astype(int)
binary_cm = confusion_matrix(y_test_binary, binary_preds)

# Group classification confusion matrix
group_preds = np.argmax(predictions[1], axis=1)
group_cm = confusion_matrix(y_test_group, group_preds)

# Specific classification confusion matrix
specific_preds = np.argmax(predictions[2], axis=1)
specific_cm = confusion_matrix(y_test, specific_preds)

# Plot confusion matrices
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plot Binary classification confusion matrix
plot_confusion_matrix(binary_cm, classes=['Goodware', 'Malicious'], title='Binary Classification Confusion Matrix')

# Plot Group classification confusion matrix
group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']
plot_confusion_matrix(group_cm, classes=group_labels, title='Group Classification Confusion Matrix')

# Plot Specific classification confusion matrix
specific_labels = ['Goodware', 'Critroni', 'CryptLocker', 'CryptoWall', 'KOLLAH', 'Kovter', 'Locker', 'MATSNU', 'PGPCODER', 'Reveton', 'TeslaCrypt', 'Trojan-Ransom']
plot_confusion_matrix(specific_cm, classes=specific_labels, title='Specific Classification Confusion Matrix')


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Generate predictions
predictions = model.predict(X_test_encoded)

# Binary predictions and metrics
binary_predictions = (predictions[0] > 0.5).astype(int)  # Convert probabilities to 0 or 1
binary_accuracy = accuracy_score(y_test_binary, binary_predictions)
binary_precision = precision_score(y_test_binary, binary_predictions)
binary_recall = recall_score(y_test_binary, binary_predictions)
binary_f1 = f1_score(y_test_binary, binary_predictions)

# Specific predictions and metrics
specific_predictions = np.argmax(predictions[2], axis=1)
specific_accuracy = accuracy_score(y_test, specific_predictions)
specific_precision = precision_score(y_test, specific_predictions, average='macro')
specific_recall = recall_score(y_test, specific_predictions, average='macro')
specific_f1 = f1_score(y_test, specific_predictions, average='macro')

# Print the results
print(f"Binary Classification Metrics:\n"
      f"Accuracy: {binary_accuracy:.4f}\n"
      f"Precision: {binary_precision:.4f}\n"
      f"Recall: {binary_recall:.4f}\n"
      f"F1 Score: {binary_f1:.4f}")

print(f"\nSpecific Classification Metrics:\n"
      f"Accuracy: {specific_accuracy:.4f}\n"
      f"Precision: {specific_precision:.4f}\n"
      f"Recall: {specific_recall:.4f}\n"
      f"F1 Score: {specific_f1:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Binary Classification Metrics:
Accuracy: 0.4033
Precision: 0.3843
Recall: 0.9231
F1 Score: 0.5427

Specific Classification Metrics:
Accuracy: 0.0295
Precision: 0.0095
Recall: 0.0608
F1 Score: 0.0151


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
# Plot the model
plot_model(model, to_file='dnn_model.png', show_shapes=True, show_layer_names=True)


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Display the model
img = mpimg.imread('dnn_model.png')
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.show()
